In [20]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 16 19:23:12 2020

@author: SOURAV
""" 
import PIL
from PIL import ImageTk, Image
from tkinter import *
import tkinter 
import cv2 
import os
import time 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import svm


protoFile = "model\\pose_deploy.prototxt"
weightsFile = "model\\pose_iter_102000.caffemodel"

nPoints = 21
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile) 
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

folderpath = "images"
root = tkinter.Tk()
root.geometry("500x500")
f = Frame()
f.pack(side=BOTTOM)
totalImages=5
 
        
def getMarkedImage(frame):
    
    scale_percent = 50 
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100) 
    dsize = (width, height) 
    frame = cv2.resize(frame, dsize)
    
    inHeight,inWidth,_ = frame.shape 

    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)
  
    net.setInput(inpBlob)
    
    output = net.forward()
    points = []
    frameHeight,frameWidth,_=frame.shape
    frameCopy=frame.copy()
    threshold=.05
    for i in range(nPoints):
    # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))
    
        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
    
            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            points.append(None)
    return frameCopy,points


def captureImage():
    i=1
    vid = cv2.VideoCapture(0)
    while i<=totalImages:
        ret, frame = vid.read() 
        
        f,p = getMarkedImage(frame.copy())
        
#         Display Pic
        frameCopy = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = ImageTk.PhotoImage(image = PIL.Image.fromarray(frameCopy))  
        canvas.create_image(100,100, image=img) 
    
        
        folderName = os.path.join(folderpath,E1.get())
        if not os.path.exists(folderName):
            os.mkdir(folderName)
        filename = os.path.join(folderName,str(i)+".jpg")
        if p.count(None)==0:
            cv2.imwrite(filename, frame)  
            var.set(str(i)+".jpg saved")
            i+=1
        else:
            var.set("None "+str(p.count(None)))
        root.update()
        time.sleep(1)
    vid.release() 

E1 = Entry(f)
E1.delete(0,END)
E1.insert(0,"1") 
E1.pack(side = LEFT)

B = tkinter.Button(f, text ="Capture", command = captureImage)
B.pack(side=LEFT)

var = StringVar()
var.set('Ready')
L1 = Label(f,textvariable = var)
L1.pack(side=LEFT)
canvas = Canvas(root, width = 300, height = 300, bg="black")      
canvas.pack(side=TOP) 



def checkImage():
    image_count=1
    while image_count<=totalImages:   
        folderName = os.path.join(folderpath,E1.get())
        if not os.path.exists(folderName):
            return
        filename = os.path.join(folderName,str(image_count)+".jpg")  
        # img = PIL.Image.open(filename)
        # w,h = img.size 
        # img = ImageTk.PhotoImage(img.resize((w//2,h//2)) )  
        
        #Detect points 
        if not os.path.exists(filename):
            print(filename,"Does not exist")
            return None,None
        frame = cv2.imread(filename)
    
        frame,p = getMarkedImage(frame) 
        displayPic(frame)
        
#         Display Pic
        frameCopy = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = ImageTk.PhotoImage(image = PIL.Image.fromarray(frameCopy))  
        canvas.create_image(100,100, image=img) 
    
        var.set(str(image_count)+".jpg Displayed") 
        image_count+=1
        root.update() 

#Feature extraction
def extractFeatures():
    signs=['left','right']
    features=None
    output=[]
    for s in signs:
        folderName = os.path.join(folderpath,s)
        for pic in os.listdir(folderName):
            print(folderName,pic)
            f,p = getMarkedImage(cv2.imread(os.path.join(folderName,pic)))
            if features is None:
                features =  np.expand_dims(np.array(p),axis=0)
            else:
                features =  np.concatenate((features,np.expand_dims(np.array(p),axis=0)),axis=0)
            output.append(s)
    np.save("data_features.npy",features)
    np.save("data_output.npy",output)
    
    
def predict():
    f= np.load("data_features.npy")
    o= np.load("data_output.npy") 
    f = f.reshape((len(o),-1))  
    
    clf = svm.SVC()
    clf.fit(f, o)
    vid = cv2.VideoCapture(0)
    while True:
        ret, frame = vid.read() 
        
        f,p = getMarkedImage(frame.copy())
        
#         Display Pic
        frameCopy = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
        img = ImageTk.PhotoImage(image = PIL.Image.fromarray(frameCopy))  
        canvas.create_image(200,200, image=img) 
     
        if p.count(None)==0: 
            features = np.array(p)
            features = features.reshape((1,-1))
            var.set( "Prediction = "+clf.predict(features)[0]) 
        else:
            var.set("None "+str(p.count(None)))
            
        root.update()
        time.sleep(1)
    vid.release() 
    
    

B = tkinter.Button(f, text ="Check", command = checkImage)
B.pack()

C = tkinter.Button(f, text ="Train", command = extractFeatures)
C.pack()


D = tkinter.Button(f, text ="Predict", command = predict)
D.pack()




root.mainloop()

Exception in Tkinter callback
Exception ignored in: <function PhotoImage.__del__ at 0x000002AE0313DF70>
Traceback (most recent call last):
  File "C:\Users\SIBSANKAR\anaconda3\lib\site-packages\PIL\ImageTk.py", line 118, in __del__
    name = self.__photo.name
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'
Traceback (most recent call last):
  File "C:\Users\SIBSANKAR\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-20-b9e22d49202a>", line 183, in predict
    img = ImageTk.PhotoImage(image = PIL.Image.fromarray(frameCopy))
  File "C:\Users\SIBSANKAR\anaconda3\lib\site-packages\PIL\ImageTk.py", line 112, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
  File "C:\Users\SIBSANKAR\anaconda3\lib\tkinter\__init__.py", line 4061, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "C:\Users\SIBSANKAR\anaconda3\lib\tkinter\__init__.py", line 3994, in __init__
    raise Runti

In [2]:
frame = cv2.imread("images\\left\\4.jpg")
frame,p = getMarkedImage(frame)

In [12]:
f= np.load("data_features.npy")
f = f.reshape((10,-1))
print(f.shape)
o= np.load("data_output.npy")
print(o)

(10, 42)
['left' 'left' 'left' 'left' 'left' 'right' 'right' 'right' 'right'
 'right']


In [13]:
from sklearn import svm
clf = svm.SVC()
clf.fit(f, o)

SVC()

In [16]:
clf.predict(f)

array(['left', 'left', 'left', 'left', 'left', 'right', 'right', 'right',
       'right', 'right'], dtype='<U5')

In [109]:
p
q=p.copy()
a = np.array(p)
b = np.array(q)
a = np.expand_dims(a,axis=0)
b = np.expand_dims(b,axis=0)
a = np.concatenate((a,b),axis=0)
np.save('data.npy', a)
a.shape

(2, 21, 2)

In [53]:
import numpy as np

a = np.array(p,dtype=(int,int))
b = a-a[0]
min,max = b.min(),b.max()
b_normal = (b-min)/(max-min)

In [112]:
a = np.array()

TypeError: array() missing required argument 'object' (pos 1)

In [10]:
data = np.load('data.npy')
data = data.reshape((-1,4))

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [62]:
b_normal

array([[0.45933014, 0.45933014],
       [0.54066986, 0.49760766],
       [0.68899522, 0.49760766],
       [0.80861244, 0.49760766],
       [0.92344498, 0.50239234],
       [0.73205742, 0.34449761],
       [0.88038278, 0.26794258],
       [0.92344498, 0.22966507],
       [1.        , 0.15789474],
       [0.68899522, 0.26794258],
       [0.80382775, 0.15789474],
       [0.88038278, 0.08133971],
       [0.92344498, 0.00478469],
       [0.61722488, 0.19617225],
       [0.72727273, 0.11483254],
       [0.77033493, 0.0430622 ],
       [0.84210526, 0.        ],
       [0.57416268, 0.15789474],
       [0.61244019, 0.08133971],
       [0.6507177 , 0.03827751],
       [0.68899522, 0.        ]])

In [69]:
pd


NameError: name 'pd' is not defined